In [ ]:
pip install selenium pandas webdriver-manager openpyxl

In [ ]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
from time import sleep
from food_list import food_set


# 페이지 다운
def page_down(num):
    store_list_area = driver.find_element(By.CSS_SELECTOR, ".Ryr1F")
    for i in range(num):
        driver.execute_script(
            "arguments[0].scrollTop += arguments[0].offsetHeight;", store_list_area
        )


food_dict = {
    "food_name": [],
    "latitude": [],
    "longitude": [],
    "address": [],
    "operating_hours": [],
    "phone_number": [],
    "website": [],
    "menu_name": [],
    "menu_price": [],
    "menu_img": [],
}
driver = wb.Chrome()

driver.get("https://map.naver.com/")

sleep(3)

search = driver.find_element(By.CSS_SELECTOR, ".input_search")
search.send_keys("호서대 음식점")
search.send_keys(Keys.ENTER)

sleep(3)

driver.switch_to.frame("searchIframe")
page_down(40)
sleep(3)

# 페이지 리스트
next_btn = driver.find_elements(By.CSS_SELECTOR, ".zRM9F> a")

# 크롤링 (페이지 리스트 만큼)
for btn in range(len(next_btn))[
    1:
]:  # next_btn[0] = 이전 페이지 버튼 무시 -> [1]부터 시작
    store_list = driver.find_elements(By.CSS_SELECTOR, ".UEzoS.rTjJo")
    title = driver.find_elements(By.CSS_SELECTOR, ".place_bluelink.TYaxT")

    for data in range(len(store_list)):  # 가게 리스트 만큼
        # 가게 리스트 iframe 전환
        driver.switch_to.default_content()
        driver.switch_to.frame("searchIframe")
        sleep(1)

        # 가게 클릭
        title[data].click()
        sleep(1)

        # 상세 페이지 iframe 전환
        driver.switch_to.default_content()
        driver.switch_to.frame("entryIframe")
        sleep(1)

        # 가게 정보
        name = driver.find_element(By.CSS_SELECTOR, ".GHAhO")
        address = driver.find_element(By.CSS_SELECTOR, ".LDgIH")

        # 로딩 기다리기
        sleep(1)

        if any(food in name.text for food in food_set):
            food_dict["food_name"].append(name.text)
            food_dict["address"].append(address.text)

    driver.switch_to.default_content()
    driver.switch_to.frame("searchIframe")

    # 다음 페이지 버튼 누를 수 없으면 종료
    if not next_btn[-1].is_enabled():
        break

    if store_list[-1]:  # 마지막 주차장일 경우 다음버튼 클릭
        next_btn[-1].click()
        sleep(2)

    else:
        print("페이지 인식 못함")
        break

driver.quit()  # 작업이 끝나면 창을 닫는다.

# 최대 길이를 기준으로 맞추기
max_length = max(len(food_dict["food_name"]), len(food_dict["address"]))

for key in food_dict:
    if len(food_dict[key]) < max_length:
        # 부족한 길이만큼 None 값을 추가
        food_dict[key].extend([None] * (max_length - len(food_dict[key])))

df = pd.DataFrame(food_dict)

# 엑셀 파일로 저장
df.to_excel("food_data.xlsx", index=False)

print("크롤링 완료 및 엑셀 파일 저장")

In [ ]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import pandas as pd
from time import sleep
from food_list import food_set

def page_down(driver, num):
    store_list_area = driver.find_element(By.CSS_SELECTOR, ".Ryr1F")
    for i in range(num):
        driver.execute_script(
            "arguments[0].scrollTop += arguments[0].offsetHeight;", store_list_area
        )
        sleep(0.5)

def get_operating_hours(driver):
    operating_hours = {}
    try:
        # 영업시간 버튼이 있는지 확인
        hours_button = driver.find_element(By.CSS_SELECTOR, "a.gKP9i.RMgN0")
        if hours_button.is_displayed():
            hours_button.click()
            sleep(1)

            days = driver.find_elements(By.CSS_SELECTOR, "span.i8cJw")
            hours = driver.find_elements(By.CSS_SELECTOR, "div.H3ua4")

            for day, hour in zip(days, hours):
                operating_hours[day.text] = hour.text.strip() or None
        else:
            print("영업시간 정보가 없습니다.")
            return None

    except NoSuchElementException:
        print("영업시간 버튼을 찾을 수 없습니다.")
        return None
    except Exception as e:
        print(f"영업시간 정보를 가져오는 중 오류 발생: {e}")
        return None

    return operating_hours if operating_hours else None

food_dict = {
    "food_name": [],
    "address": [],
    "operating_hours": [],
    "phone_number": [],
    "website": [],
}

driver = wb.Chrome()
driver.get("https://map.naver.com/")

sleep(3)

search = driver.find_element(By.CSS_SELECTOR, ".input_search")
search.send_keys("호서대 음식점")
search.send_keys(Keys.ENTER)

sleep(3)

driver.switch_to.frame("searchIframe")
page_down(driver, 40)
sleep(3)

next_btn = driver.find_elements(By.CSS_SELECTOR, ".zRM9F> a")

store_count = 0

for btn in range(len(next_btn))[1:]:
    store_list = driver.find_elements(By.CSS_SELECTOR, ".UEzoS.rTjJo")
    title = driver.find_elements(By.CSS_SELECTOR, ".place_bluelink.TYaxT")

    for data in range(len(store_list)):
        if store_count >= 10:
            break

        try:
            driver.switch_to.default_content()
            driver.switch_to.frame("searchIframe")
            sleep(1)

            title[data].click()
            sleep(2)

            driver.switch_to.default_content()
            driver.switch_to.frame("entryIframe")
            sleep(2)

            name = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".GHAhO"))
            ).text
            
            try:
                address = driver.find_element(By.CSS_SELECTOR, ".LDgIH").text
            except NoSuchElementException:
                address = None

            if any(food in name for food in food_set):
                operating_hours = get_operating_hours(driver)

                try:
                    phone_number = driver.find_element(By.CSS_SELECTOR, "span.xlx7Q").text
                except NoSuchElementException:
                    phone_number = None

                try:
                    website = driver.find_element(By.CSS_SELECTOR, "a.CHmqa").get_attribute("href")
                except NoSuchElementException:
                    website = None

                food_dict["food_name"].append(name)
                food_dict["address"].append(address)
                food_dict["operating_hours"].append(operating_hours)
                food_dict["phone_number"].append(phone_number)
                food_dict["website"].append(website)

                store_count += 1
                print(f"{store_count}번째 음식점 정보 크롤링 완료: {name}")

                if store_count >= 10:
                    break

        except Exception as e:
            print(f"데이터 추출 중 오류 발생: {e}")
            continue

    if store_count >= 10:
        break

    driver.switch_to.default_content()
    driver.switch_to.frame("searchIframe")

    if not next_btn[-1].is_enabled():
        break

    if store_list[-1]:
        next_btn[-1].click()
        sleep(2)
    else:
        print("페이지 인식 못함")
        break

driver.quit()

df = pd.DataFrame(food_dict)

# 비어있는 값들을 NULL로 채우기
df = df.fillna("NULL")

df.to_excel("food_data_10stores.xlsx", index=False)

print("크롤링 완료 및 엑셀 파일 저장")

In [ ]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
import pandas as pd
from time import sleep
from food_list import food_set

def page_down(driver, num):
    store_list_area = driver.find_element(By.CSS_SELECTOR, ".Ryr1F")
    for i in range(num):
        driver.execute_script(
            "arguments[0].scrollTop += arguments[0].offsetHeight;", store_list_area
        )
        sleep(0.5)

def get_operating_hours(driver):
    operating_hours = {}
    try:
        hours_button = driver.find_element(By.CSS_SELECTOR, "a.gKP9i.RMgN0")
        if hours_button.is_displayed():
            hours_button.click()
            sleep(1)

            days = driver.find_elements(By.CSS_SELECTOR, "span.i8cJw")
            hours = driver.find_elements(By.CSS_SELECTOR, "div.H3ua4")

            for day, hour in zip(days, hours):
                operating_hours[day.text] = hour.text.strip() or None
        else:
            print("영업시간 정보가 없습니다.")
            return None

    except NoSuchElementException:
        print("영업시간 버튼을 찾을 수 없습니다.")
        return None
    except Exception as e:
        print(f"영업시간 정보를 가져오는 중 오류 발생: {e}")
        return None

    return operating_hours if operating_hours else None

food_dict = {
    "food_name": [],
    "address": [],
    "operating_hours": [],
    "phone_number": [],
    "website": [],
}

driver = wb.Chrome()
driver.get("https://map.naver.com/")

sleep(3)

search = driver.find_element(By.CSS_SELECTOR, ".input_search")
search.send_keys("호서대 음식점")
search.send_keys(Keys.ENTER)

sleep(3)

store_count = 0
page_num = 1
last_page_reached = False

while not last_page_reached:
    try:
        driver.switch_to.default_content()
        driver.switch_to.frame("searchIframe")
        page_down(driver, 40)
        sleep(3)

        store_list = driver.find_elements(By.CSS_SELECTOR, ".UEzoS.rTjJo")
        title = driver.find_elements(By.CSS_SELECTOR, ".place_bluelink.TYaxT")

        for data in range(len(store_list)):
            try:
                driver.switch_to.default_content()
                driver.switch_to.frame("searchIframe")
                sleep(1)

                # StaleElementReferenceException 방지를 위해 매번 요소를 다시 찾음
                store_list = driver.find_elements(By.CSS_SELECTOR, ".UEzoS.rTjJo")
                title = driver.find_elements(By.CSS_SELECTOR, ".place_bluelink.TYaxT")
                title[data].click()
                sleep(2)

                driver.switch_to.default_content()
                driver.switch_to.frame("entryIframe")
                sleep(2)

                name = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, ".GHAhO"))
                ).text
                
                try:
                    address = driver.find_element(By.CSS_SELECTOR, ".LDgIH").text
                except NoSuchElementException:
                    address = None

                if any(food in name for food in food_set):
                    operating_hours = get_operating_hours(driver)

                    try:
                        phone_number = driver.find_element(By.CSS_SELECTOR, "span.xlx7Q").text
                    except NoSuchElementException:
                        phone_number = None

                    try:
                        website = driver.find_element(By.CSS_SELECTOR, "a.CHmqa").get_attribute("href")
                    except NoSuchElementException:
                        website = None

                    food_dict["food_name"].append(name)
                    food_dict["address"].append(address)
                    food_dict["operating_hours"].append(operating_hours)
                    food_dict["phone_number"].append(phone_number)
                    food_dict["website"].append(website)

                    store_count += 1
                    print(f"{store_count}번째 음식점 정보 크롤링 완료: {name}")

            except StaleElementReferenceException:
                print("페이지가 새로고침되었습니다. 다음 항목으로 넘어갑니다.")
                continue
            except Exception as e:
                print(f"데이터 추출 중 오류 발생: {e}")
                continue

        # 다음 페이지로 이동
        driver.switch_to.default_content()
        driver.switch_to.frame("searchIframe")
        next_btn = driver.find_elements(By.CSS_SELECTOR, ".zRM9F> a")

        if len(next_btn) > 1 and next_btn[-1].is_enabled():
            current_page = driver.find_element(By.CSS_SELECTOR, "a.qxokY").text
            next_btn[-1].click()
            sleep(2)
            
            # 페이지 변경 확인
            WebDriverWait(driver, 10).until(
                lambda d: d.find_element(By.CSS_SELECTOR, "a.qxokY").text != current_page
            )
            
            page_num += 1
            print(f"페이지 {page_num}로 이동")
        else:
            print("마지막 페이지에 도달했습니다.")
            last_page_reached = True

    except Exception as e:
        print(f"페이지 이동 중 오류 발생: {e}")
        break

driver.quit()

df = pd.DataFrame(food_dict)

# 비어있는 값들을 NULL로 채우기
df = df.fillna("NULL")

df.to_excel("food_data_all_stores.xlsx", index=False)

print(f"크롤링 완료 및 엑셀 파일 저장. 총 {store_count}개의 음식점 정보가 수집되었습니다.")

In [ ]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
import pandas as pd
from time import sleep
from food_list import food_set
import re

def page_down(driver, num):
    store_list_area = driver.find_element(By.CSS_SELECTOR, ".Ryr1F")
    for i in range(num):
        driver.execute_script(
            "arguments[0].scrollTop += arguments[0].offsetHeight;", store_list_area
        )
        sleep(0.5)

def get_operating_hours(driver):
    operating_hours = {}
    try:
        hours_button = driver.find_element(By.CSS_SELECTOR, "a.gKP9i.RMgN0")
        if hours_button.is_displayed():
            hours_button.click()
            sleep(1)

            days = driver.find_elements(By.CSS_SELECTOR, "span.i8cJw")
            hours = driver.find_elements(By.CSS_SELECTOR, "div.H3ua4")

            for day, hour in zip(days, hours):
                operating_hours[day.text] = hour.text.strip() or None
        else:
            print("영업시간 정보가 없습니다.")
            return None

    except NoSuchElementException:
        print("영업시간 버튼을 찾을 수 없습니다.")
        return None
    except Exception as e:
        print(f"영업시간 정보를 가져오는 중 오류 발생: {e}")
        return None

    return operating_hours if operating_hours else None

food_dict = {
    "food_name": [],
    "address": [],
    "latitude": [],
    "longitude": [],
    "operating_hours": [],
    "phone_number": [],
    "website": [],
}

driver = wb.Chrome()
driver.get("https://map.naver.com/")

sleep(3)

search = driver.find_element(By.CSS_SELECTOR, ".input_search")  
search.send_keys("호서대 음식점")
search.send_keys(Keys.ENTER)

sleep(3)

store_count = 0
max_stores = 5

try:
    driver.switch_to.default_content()
    driver.switch_to.frame("searchIframe")
    page_down(driver, 40)
    sleep(3)

    store_list = driver.find_elements(By.CSS_SELECTOR, ".UEzoS.rTjJo")  
    title = driver.find_elements(By.CSS_SELECTOR, ".place_bluelink.TYaxT")

    for data in range(len(store_list)):
        if store_count >= max_stores:
            break

        try:
            driver.switch_to.default_content()
            driver.switch_to.frame("searchIframe")
            sleep(1)

            store_list = driver.find_elements(By.CSS_SELECTOR, ".UEzoS.rTjJo")
            title = driver.find_elements(By.CSS_SELECTOR, ".place_bluelink.TYaxT")
            title[data].click()
            sleep(2)

            driver.switch_to.default_content()
            driver.switch_to.frame("entryIframe")
            sleep(2)

            name = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".GHAhO"))
            ).text
            
            try:
                address = driver.find_element(By.CSS_SELECTOR, ".LDgIH").text
            except NoSuchElementException:
                address = None
            
            # 위도, 경도 정보 추출
            script_content = driver.find_element(By.XPATH, '//script[contains(., "window.__APOLLO_STATE__")]').get_attribute('innerHTML')
            match = re.search(r'"x":"([\d.]+)","y":"([\d.]+)"', script_content)
            if match:
                longitude = match.group(1)
                latitude = match.group(2)
            else:
                longitude, latitude = None, None

            if any(food in name for food in food_set):
                operating_hours = get_operating_hours(driver)
                
                try:
                    phone_number = driver.find_element(By.CSS_SELECTOR, "span.xlx7Q").text
                except NoSuchElementException:
                    phone_number = None
                    
                try:
                    website = driver.find_element(By.CSS_SELECTOR, "a.CHmqa").get_attribute("href")  
                except NoSuchElementException:
                    website = None
                    
                food_dict["food_name"].append(name)
                food_dict["address"].append(address)
                food_dict["latitude"].append(latitude)
                food_dict["longitude"].append(longitude)  
                food_dict["operating_hours"].append(operating_hours)
                food_dict["phone_number"].append(phone_number)
                food_dict["website"].append(website)
                
                store_count += 1
                print(f"{store_count}번째 음식점 정보 크롤링 완료: {name}")
                
        except StaleElementReferenceException:
            print("페이지가 새로고침되었습니다. 다음 항목으로 넘어갑니다.")
            continue
        except Exception as e:
            print(f"데이터 추출 중 오류 발생: {e}")
            continue
            
except Exception as e:
    print(f"크롤링 중 오류 발생: {e}")

driver.quit()

df = pd.DataFrame(food_dict)
df = df.fillna("NULL")

df.to_excel("food_data_5_stores.xlsx", index=False)

print(f"크롤링 완료 및 엑셀 파일 저장. 총 {store_count}개의 음식점 정보가 수집되었습니다.")

In [ ]:
# 임시 완성본
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
import pandas as pd
from time import sleep
from food_list import food_set
import re

def page_down(driver, num):
    store_list_area = driver.find_element(By.CSS_SELECTOR, ".Ryr1F")
    for i in range(num):
        driver.execute_script(
            "arguments[0].scrollTop += arguments[0].offsetHeight;", store_list_area
        )
        sleep(0.5)

def get_operating_hours(driver):
    operating_hours = {}
    try:
        hours_button = driver.find_element(By.CSS_SELECTOR, "a.gKP9i.RMgN0")
        if hours_button.is_displayed():
            hours_button.click()
            sleep(1)

            days = driver.find_elements(By.CSS_SELECTOR, "span.i8cJw")
            hours = driver.find_elements(By.CSS_SELECTOR, "div.H3ua4")

            for day, hour in zip(days, hours):
                operating_hours[day.text] = hour.text.strip() or None
        else:
            print("영업시간 정보가 없습니다.")
            return None

    except NoSuchElementException:
        print("영업시간 버튼을 찾을 수 없습니다.")
        return None
    except Exception as e:
        print(f"영업시간 정보를 가져오는 중 오류 발생: {e}")
        return None

    return operating_hours if operating_hours else None

food_dict = {
    "food_name": [],
    "address": [],
    "latitude": [],
    "longitude": [],
    "operating_hours": [],
    "phone_number": [],
    "website": [],
}

driver = wb.Chrome()
driver.get("https://map.naver.com/")

sleep(3)

search = driver.find_element(By.CSS_SELECTOR, ".input_search")
search.send_keys("호서대 음식점")
search.send_keys(Keys.ENTER)

sleep(3)

store_count = 0
page_num = 1
last_page_reached = False

while not last_page_reached:
    try:
        driver.switch_to.default_content()
        driver.switch_to.frame("searchIframe")
        page_down(driver, 40)
        sleep(3)

        store_list = driver.find_elements(By.CSS_SELECTOR, ".UEzoS.rTjJo")
        title = driver.find_elements(By.CSS_SELECTOR, ".place_bluelink.TYaxT")

        for data in range(len(store_list)):
            try:
                driver.switch_to.default_content()
                driver.switch_to.frame("searchIframe")
                sleep(1)

                store_list = driver.find_elements(By.CSS_SELECTOR, ".UEzoS.rTjJo")
                title = driver.find_elements(By.CSS_SELECTOR, ".place_bluelink.TYaxT")
                title[data].click()
                sleep(2)

                driver.switch_to.default_content()
                driver.switch_to.frame("entryIframe")
                sleep(2)

                name = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, ".GHAhO"))
                ).text
                
                try:
                    address = driver.find_element(By.CSS_SELECTOR, ".LDgIH").text
                except NoSuchElementException:
                    address = None
                
                script_content = driver.find_element(By.XPATH, '//script[contains(., "window.__APOLLO_STATE__")]').get_attribute('innerHTML')
                match = re.search(r'"x":"([\d.]+)","y":"([\d.]+)"', script_content)
                if match:
                    longitude = match.group(1)
                    latitude = match.group(2)
                else:
                    longitude, latitude = None, None

                if any(food in name for food in food_set):
                    operating_hours = get_operating_hours(driver)
                    
                    try:
                        phone_number = driver.find_element(By.CSS_SELECTOR, "span.xlx7Q").text
                    except NoSuchElementException:
                        phone_number = None
                        
                    try:
                        website = driver.find_element(By.CSS_SELECTOR, "a.CHmqa").get_attribute("href")
                    except NoSuchElementException:
                        website = None
                        
                    food_dict["food_name"].append(name)
                    food_dict["address"].append(address)
                    food_dict["latitude"].append(latitude)
                    food_dict["longitude"].append(longitude)
                    food_dict["operating_hours"].append(operating_hours)
                    food_dict["phone_number"].append(phone_number)
                    food_dict["website"].append(website)
                    
                    store_count += 1
                    print(f"{store_count}번째 음식점 정보 크롤링 완료: {name}")
                    
            except StaleElementReferenceException:
                print("페이지가 새로고침되었습니다. 다음 항목으로 넘어갑니다.")
                continue
            except Exception as e:
                print(f"데이터 추출 중 오류 발생: {e}")
                continue

        # 다음 페이지로 이동
        driver.switch_to.default_content()
        driver.switch_to.frame("searchIframe")
        next_btn = driver.find_elements(By.CSS_SELECTOR, ".zRM9F> a")

        if len(next_btn) > 1 and next_btn[-1].is_enabled():
            current_page = driver.find_element(By.CSS_SELECTOR, "a.qxokY").text
            next_btn[-1].click()
            sleep(2)
            
            # 페이지 변경 확인
            WebDriverWait(driver, 10).until(
                lambda d: d.find_element(By.CSS_SELECTOR, "a.qxokY").text != current_page
            )
            
            page_num += 1
            print(f"페이지 {page_num}로 이동")
        else:
            print("마지막 페이지에 도달했습니다.")
            last_page_reached = True

    except Exception as e:
        print(f"페이지 이동 중 오류 발생: {e}")
        break

driver.quit()

df = pd.DataFrame(food_dict)
df = df.fillna("NULL")

df.to_excel("food_data_stores.xlsx", index=False)

print(f"크롤링 완료 및 엑셀 파일 저장. 총 {store_count}개의 음식점 정보가 수집되었습니다.")